In [6]:
import numpy as np
import pandas as pd
import random
import math
import cvxpy

class contract:
    def __init__(self, numRows, seed=76):
        self.numRows = numRows
        self.seed = seed
        self.df = self.genDF ()
        self.star = self.finalStar ()
        
              
    def getWeights (self):
        np.random.seed(self.seed) 
        numRows = self.numRows
        weights = np.random.random(numRows)+0.1
        weights /= weights.sum()
        weights = np.round(weights,4)
        self.seed=self.seed+1
        return weights

    def genDF(self):
        random.seed(self.seed) 
        numRows = self.numRows   
        measureNames = list(map(lambda x: 'Measure%d'%x,range(numRows)))
        scores = [random.randint(40,100) for p in range(0, numRows)]
        weights = self.getWeights()
        lookUp = {40:1,50:1.5,60:2,70:2.5,80:3,90:3.5,100:4}
        ratings = list(map(lambda x: lookUp[(int(math.ceil(x / 10.0)) * 10)],scores))
        ease = self.getWeights ()
        cost = [random.randint(1000,10000) for p in range(0, numRows)]
        df = pd.DataFrame([measureNames,scores,weights,ratings,ease,cost]).T
        df.columns=['Measure','Score','Weight','Rating','Ease','Cost']
        return df
    
    def finalStar (self):
        df=self.df
        return np.round((df.Rating*df.Weight).sum(),2)





#simulate a contract with n measures

difficultyThreshold=0.75
costThreshold=0.75
numberofMeasures=40

contract1=contract(numberofMeasures, seed=14)
df=contract1.df
currentRating=round(contract1.star*2)/2

#constants
weights = np.array(df.Weight)
cost= np.array(df.Cost)
ease = (np.clip(df.Ease, 0.0001, 0.9999))
difficulty = np.array(-1*np.log(list(ease))) #turn ease into difficulty
ratings= np.array(df.Rating)
D=difficulty.sum()*difficultyThreshold #difficulty of selected measures should be 75% of total difficulty
C=cost.sum()*costThreshold # cost of selected measures should be 75% of total cost

#parameter
selection = cvxpy.Variable(len(weights),integer=True)


#constraints
recommendedRating = cvxpy.sum((ratings+selection)*weights)
difficultyConstraint = difficulty * selection <= D
costConstraint = cost * selection <= C
starConstraint= recommendedRating-currentRating >= 0.5
ratingConstraint = ratings + selection <= 5
globalConstraint = recommendedRating <= 5 
selectionConstraint1 = selection <= 5
selectionConstraint2 = selection >= 0
selectionConstraint3 = cvxpy.sum(cvxpy.ceil(cvxpy.log(selection)))>=0
delta=np.finfo(np.float32).eps

regularizer=0.2*cvxpy.sum(selection)

objectiveFunction=recommendedRating-regularizer
constraints = [difficultyConstraint,costConstraint,starConstraint,ratingConstraint, globalConstraint,\
              selectionConstraint1,selectionConstraint2]

#objective function
starsProblem = cvxpy.Problem(cvxpy.Maximize(objectiveFunction),constraints)

# Solving the problem
starsProblem.solve(solver=cvxpy.GLPK_MI)
roundedRating=np.round(recommendedRating.value*2)/2

focusMeasureIndex=[i for i, x in enumerate(selection.value) if x >= 1]

totalCost=(cost[focusMeasureIndex]*selection.value[focusMeasureIndex]).sum()
totalDifficulty=(difficulty[focusMeasureIndex]*selection.value[focusMeasureIndex]).sum()

print ('Total cost of Solution: $%.0f (%.0f%% of Budget)'%(totalCost,totalCost/(costThreshold*cost.sum())*100))
print ('Total ease of Solution: %.0f (%.0f%% of Difficulty Threshold)'%\
       (totalDifficulty,totalDifficulty/(difficultyThreshold*difficulty.sum())*100))


print ('Recommended rating: %.1f' %roundedRating)
print ('Current rating: %.1f'%currentRating)
print ('# of Recommended measures: %.0f' %((selection.value>0).sum()))
dfRec=df[df.Measure.isin(list(df.Measure[focusMeasureIndex]))]
dfRec=dfRec.assign(RecRating=dfRec.Rating+selection.value[focusMeasureIndex])
display (dfRec)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
0.0 <= Sum(ceil(log(var492)), None, False) , because the following subexpressions are not:
|--  ceil(log(var492))
However, the problem does follow DQCP rules. Consider calling solve() with `qcp=True`.

In [7]:
cvxpy.s(selection)

AttributeError: module 'cvxpy' has no attribute 'mean'